In [2]:
pip install pandas


Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install spotipy

Note: you may need to restart the kernel to use updated packages.


In [4]:
%pip install urllib3 --upgrade 
%pip install requests --upgrade 
%pip install spotipy --upgrade
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import os
import numpy as np
import timeit
import pandas as pd
from operator import itemgetter
import datetime as dt
from collections import Counter
from operator import itemgetter
from itertools import chain
import json

os.environ['SPOTIPY_CLIENT_ID'] = '3191e7e8b04e46c1af64f49bcdd020be'
os.environ['SPOTIPY_CLIENT_SECRET'] = '88dc925eff0d4006bb6d0b9b2401ac59'
auth_manager = SpotifyClientCredentials()
sp = spotipy.Spotify(auth_manager=auth_manager)

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [5]:
# calculate artist concentration value for a playlist
# low value means high concentration - maybe rename to artist diversity?
# input: 2D array of artists for each song
# output: real value between 0 and 1
def get_artist_div(artists):
    unique_artists = len(set(artists))
    total_appearances = len(artists)
    artist_div = unique_artists / total_appearances
    return artist_div

MS_IN_DAY = 86400
def get_artists_and_dates(uris):
    # do the batching above this- here we assume that there are no more than 50 URIs
    # i could put an assert statement here but it would slow the code down
    # assert(len(uris) <= 50)
    artists = {}
    dates = {}
    tracks = sp.tracks(uris)['tracks']
    none_uris = []
    for uri, track in zip(uris, tracks):
        if track is None:
            print(track)
            none_uris.append(uri)
        else:
            artists[uri] = pd.DataFrame(track['artists'])['name'].values.tolist()

            date = track['album']['release_date']
            while len(date) < 10:
                date += '-01'
            ms_date = dt.datetime.timestamp(dt.datetime.strptime(date,"%Y-%m-%d"))
            dates[uri] = ms_date / MS_IN_DAY
            
    return artists, dates, none_uris

# get all audio features for a given playlist
def get_playlist_audio_features_batch(playlist_uris):
    remainder = len(playlist_uris) % 100
    all_features = []
    if remainder > 0: 
        all_features = sp.audio_features(playlist_uris[:remainder])
    while remainder < len(playlist_uris):
        all_features += sp.audio_features(playlist_uris[remainder:remainder+100])
        remainder += 100
    
    none_uris = []
    remainder = len(playlist_uris) % 50
    artists, dates = {}, {}
    if remainder > 0:
        artists, dates, none_uris_iter = get_artists_and_dates(playlist_uris[:remainder])
        none_uris += none_uris_iter
    while remainder < len(playlist_uris):
        next_artists, next_dates, none_uris_iter = get_artists_and_dates(playlist_uris[remainder:remainder+50])

        artists.update(next_artists)
        dates.update(next_dates)
        remainder += 50
        none_uris += none_uris_iter

    return all_features, artists, dates, none_uris

# splits out uris that we have already called the api for
# also eliminates duplicate songs within a playlist
def deduplicate_uris(playlist_uris, used_tracks_by_uri):
    duplicates = np.intersect1d(playlist_uris, np.array(list(used_tracks_by_uri.keys())))
    new_uris = np.setdiff1d(playlist_uris, np.array(list(used_tracks_by_uri.keys())))
    return duplicates, new_uris

# calculate aggregate audio features for a playlist
# currently only focused on audio features, I'll deal with artist and release date later
def get_playlist_aggregate_features(tracks, used_tracks_by_uri, used_artists_by_uri, used_dates_by_uri):

    tracks = pd.DataFrame(tracks)
    duplicates, new_uris = deduplicate_uris(tracks['track_uri'], used_tracks_by_uri) # deduplicate here

    all_features, artists, dates, none_uris = get_playlist_audio_features_batch(new_uris)

    # add duplicates to this playlist for aggregate metric calculation
    dup_artists = []
    dup_dates = []
    if len(duplicates) > 1:
        all_features += list(itemgetter(*duplicates)(used_tracks_by_uri))
        dup_artists = list(itemgetter(*duplicates)(used_artists_by_uri)) # artists is a dict - update it accordingly
        dup_dates = list(itemgetter(*duplicates)(used_dates_by_uri))
    elif len(duplicates) == 1:
        all_features.append(used_tracks_by_uri[duplicates[0]])
        dup_artists = used_artists_by_uri[duplicates[0]]
        dup_dates = [used_dates_by_uri[duplicates[0]]]

    all_features = pd.DataFrame(all_features)
    
    # update list of already used tracks
    for i, uri in zip(range(len(new_uris)), new_uris):
        if uri not in none_uris:
            used_tracks_by_uri[uri] = all_features.loc[all_features['uri'] == uri].to_dict('records')[0]
            used_artists_by_uri[uri] = artists[uri]
            used_dates_by_uri[uri] = dates[uri]


    # calculate aggregate features
    feature_list = ['danceability', 'energy', 'loudness', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']
    aggregate_features = {}
    for feature in feature_list:
        all_by_feature = all_features[feature]
        aggregate_features[feature] = np.mean(all_by_feature)
        aggregate_features[feature+'_var'] = np.var(all_by_feature)

    aggregate_features['date'] = np.mean(list(dates.values()) + dup_dates)
    aggregate_features['date_var'] = np.var(list(dates.values()) + dup_dates)

    if (len(dup_dates) > 1):
        dup_artists = list(chain(*dup_artists))
    aggregate_features['artist_div'] = get_artist_div(list(chain(*artists.values())) + dup_artists)
    return aggregate_features

# build the aggregate dataset!
def build_dataset_slice(playlists):
    aggregate_dataset = []
    used_tracks_by_uri = {}
    used_artists_by_uri = {}
    used_dates_by_uri = {}
    for playlist in playlists:
        aggregate_dataset.append(get_playlist_aggregate_features(playlist['tracks'], used_tracks_by_uri, used_artists_by_uri, used_dates_by_uri))
    return aggregate_dataset

In [6]:
slice_prefix = 'mpd_slice_'
slice_zeros = '000_'
slice_suffix = '999.json'

db_prefix = 'zdb_slice_'
db_zeros = '000_'
db_suffix = '999.csv'

In [ ]:
NUM_SLICES = 2
for i in range(NUM_SLICES):
    print('Starting slice', i)
    with open(slice_prefix + i + slice_zeros + i + slice_suffix) as json_file:
        data = json.load(json_file)
    print('Opened file for slice', i)
    dataset_slice = pd.DataFrame(build_dataset_slice(data['playlists']))
    print('Built dataset for slice', i)
    dataset_slice.to_csv(r''+db_prefix + i + db_zeros + i + db_suffix)
    print('Completed slice', i)
